## DIA 032: Implementación de Pruebas de Carga y Rendimiento con Locust

Introducción a Locust:

Locust es una herramienta de pruebas de carga de código abierto que te permite definir escenarios de usuario en Python.
Permite simular cientos o miles de usuarios concurrentes para medir el rendimiento de tus endpoints.
Definición de Escenarios de Prueba:

Se creó un archivo de configuración (locustfile.py) que define tareas para simular acciones comunes, como iniciar sesión, enviar solicitudes al endpoint de predicción y acceder a otros endpoints críticos.
Los escenarios incluyen tanto endpoints protegidos (que requieren autenticación) como endpoints públicos, lo que ayuda a obtener una visión completa del comportamiento de la API.
Medición y Reporte de Resultados:

Locust genera reportes en tiempo real que muestran métricas como el número de solicitudes por segundo, el tiempo de respuesta medio, el porcentaje de errores y otros indicadores clave.
Estas métricas permiten identificar rápidamente posibles problemas de rendimiento y ajustar la configuración o la infraestructura según sea necesario.
Beneficios Obtenidos:

Detección de Cuellos de Botella: Permite identificar áreas de la API que requieren optimización para manejar mejor el tráfico elevado.
Validación de Escalabilidad: Ayuda a confirmar que la aplicación puede escalar de manera adecuada en un entorno de producción.
Optimización de Recursos: Ofrece datos para ajustar la configuración del servidor y mejorar la eficiencia en el uso de recursos.
Recomendaciones para Continuar:

Ajuste de Límites: Utiliza los resultados de las pruebas de carga para refinar las políticas de rate limiting y los recursos asignados.
Integración en el Pipeline: Considera integrar las pruebas de carga en tu pipeline de CI/CD para detectar de manera temprana problemas de rendimiento tras cada cambio en el código.
Monitoreo Continuo: Complementa las pruebas de carga con herramientas de monitoreo (como Prometheus y Grafana) para obtener una visión en tiempo real del comportamiento de la API bajo carga.

A continuación se muestra un ejemplo completo del archivo locustfile.py para realizar pruebas de carga y rendimiento con Locust. Este script define escenarios de usuario que simulan acciones comunes en tu API, como iniciar sesión, enviar solicitudes de predicción y, opcionalmente, registrar nuevos usuarios.

python
Copiar
# locustfile.py

from locust import HttpUser, task, between
import json
import os

class MNISTUser(HttpUser):
    # Tiempo de espera entre tareas (entre 1 y 5 segundos)
    wait_time = between(1, 5)

    def on_start(self):
        """
        Esta función se ejecuta al iniciar cada usuario virtual.
        Aquí se simula el proceso de login para obtener el token de acceso JWT.
        """
        # Definir las credenciales de prueba; asegúrate de que estos usuarios existan en la base de datos o ajusta según tus pruebas
        login_data = {"username": "testuser", "password": "testpassword"}
        with self.client.post("/login", json=login_data, catch_response=True) as response:
            if response.status_code == 200:
                data = response.json()
                self.access_token = data["access_token"]
                # Preparar el header para solicitudes autenticadas
                self.headers = {"Authorization": f"Bearer {self.access_token}"}
            else:
                # Si el login falla, registrar el error y marcar la respuesta como fallida
                response.failure("Login fallido, asegúrate de que el usuario de prueba exista")

    @task(2)
    def predict(self):
        """
        Tarea para simular una solicitud de predicción.
        Se envía una imagen de prueba al endpoint '/predict'.
        """
        # Asumimos que existe un archivo de prueba llamado 'test.png' en el mismo directorio que este script.
        # Asegúrate de tener ese archivo o ajusta la ruta a un archivo válido.
        file_path = "test.png"
        if not os.path.exists(file_path):
            # Si el archivo no existe, salimos de la tarea
            self.environment.runner.quit()
            return

        with open(file_path, "rb") as image_file:
            files = {"file": ("test.png", image_file, "image/png")}
            # Realizar la solicitud POST al endpoint '/predict' usando el token de acceso
            self.client.post("/predict", headers=self.headers, files=files)

    @task(1)
    def login_again(self):
        """
        Tarea opcional para simular múltiples intentos de login.
        Esto es útil para probar la robustez del endpoint de login y las políticas de rate limiting.
        """
        login_data = {"username": "testuser", "password": "testpassword"}
        self.client.post("/login", json=login_data)

    @task(1)
    def register(self):
        """
        Tarea opcional para simular el registro de nuevos usuarios.
        Cada registro utiliza un nombre de usuario y correo electrónico únicos.
        """
        import time
        unique_id = int(time.time() * 1000)
        new_user = {
            "username": f"user_{unique_id}",
            "email": f"user_{unique_id}@example.com",
            "password": "testpassword"
        }
        self.client.post("/register", json=new_user)

Explicación del Código:
Clase MNISTUser (HttpUser):

Define un usuario virtual que simula acciones en la API.
La propiedad wait_time establece un tiempo de espera aleatorio entre 1 y 5 segundos entre cada tarea, simulando el comportamiento real de un usuario.
Método on_start:

Se ejecuta cuando cada usuario virtual inicia.
Realiza una solicitud POST al endpoint /login con credenciales predefinidas.
Si el login es exitoso, extrae el token JWT y configura el header Authorization para futuras solicitudes.
Tarea predict:

Simula la solicitud de predicción enviando un archivo de imagen al endpoint /predict.
Abre el archivo test.png y lo envía utilizando la opción files de la solicitud POST.
Requiere que el archivo test.png exista en el mismo directorio que el locustfile.py. Puedes ajustar la ruta o nombre del archivo según tus necesidades.
Tarea login_again:

Simula intentos adicionales de login para probar el endpoint de login y sus límites (rate limiting).
Tarea register:

Simula el registro de nuevos usuarios.
Genera un nombre de usuario y correo electrónico únicos utilizando la marca de tiempo actual.
Envía la solicitud al endpoint /register.
Ejecutar la Prueba de Carga con Locust:
Asegúrate de tener Locust instalado:

bash
Copiar
pip install locust
Ejecuta Locust:

bash
Copiar
locust -f locustfile.py
Accede a la Interfaz Web de Locust:

Abre un navegador y ve a http://localhost:8089. Desde allí, configura el número de usuarios y la tasa de generación de nuevos usuarios para comenzar la prueba de carga.

Este código te permite simular el comportamiento de usuarios reales en tu API, ayudándote a identificar cuellos de botella y validar la escalabilidad y rendimiento del sistema en condiciones de alto tráfico.